# Kaggle Challenge: Detección de elementos patrimoniales





In [1]:
import tensorflow as tf
from keras import optimizers
from keras.layers.normalization import BatchNormalization
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten,Conv2D, MaxPooling2D,LeakyReLU
from pathlib import Path
from PIL import Image
from keras.utils import to_categorical, plot_model
from keras.callbacks import EarlyStopping, ModelCheckpoint


Using TensorFlow backend.







































# Funciones a utilizar

*image_set_generator* Es una función que toma cada imagen, la transforma en un arreglo tri-dimensional y luego la almacena en una lista que posteriormente es transformada en arreglo para su posterior entrenamiento o prediccion según seal el caso.

Parámetros:

X: Corresponde al arreglo que contenga los nombres de las imagenes a utilizar.
image_path: Corresponde a un string que contiene la direccion de donde se encuentran (carpeta)
image_format: La extensión del archivo, se añadio para poder recibir más tipos de imagenes.

In [2]:
def image_set_generator(X,image_path,image_format):
    gen = []
    for i in X:
        image =  tf.keras.preprocessing.image.load_img(image_path+i+'.'+image_format,target_size=None)
        data =keras.preprocessing.image.img_to_array(image)
        gen.append(data)
       
    return np.asarray(gen)

cat_encode/decode: Este par de funciones transforma el arreglo de etiquetas en categorias binarias para mejorar así su representación y evitar problemas de entrenamiento del modelo. 

Parámetros:

Data: informacion a transformar o revertir transformacion (lista o arreglo)
cat: conjunto de categorias (lista o arreglo)

In [3]:
def cat_encode(data,cat):
    data=list(data)
    cat=list(cat)
    for i in range(len(data)):
        for j in range(len(cat)):
            if data[i]==cat[j]:
                data[i]=j
                break
            else:
                continue
    return keras.utils.to_categorical(data,len(cat))
    #return np.asarray(data)
def cat_decode(data,cat):
    #Y=data
    Y= [np.argmax(y, axis=None, out=None) for y in data]
    cat=list(cat)
    
    for i in range(len(Y)):
        Y[i]=cat[Y[i]]
    return Y

Autopredict automatiza el proceso de prediccion y guardado en un archivo csv. Además incorpora una opcion para mostrar resultados

auto_predict(data_x,data_y,cat,example_show):

data_x: corresponde al conjunto de datos que se busca predecir (X_train)
data_set: corresponde al conjunto de datos que se ingresará en el archivo csv
cat:categorias
example_show: True/False si se quiere adjuntar un muestreo de los primeros 20 resultados de la prediccion.

In [4]:
def auto_predict(data_x,data_set,cat,example_show):
    pred = model.predict(data_x)
    y_test = cat_decode(pred,cat)
    data_set['Expected']= y_test
    data_set.to_csv('sample_submission_1234.csv',columns=['Id','Expected'],index=False)
    if example_show==True:
        return print("Exito!\n",y_test[0:20])
    else:
        return print("Exito")

# Cargando la data

In [5]:
train_labels = pd.read_csv('./data/train_labels.csv')
test_labels = pd.read_csv('./data/sample_submission.csv')
train_path = './data/train_images/'
test_path = './data/test_images/'
X_names = train_labels["Id"].values
X_train = image_set_generator(X_names,train_path,'jpg')


Y_train = train_labels["Expected"]
cat = Y_train.unique()
Y_train=cat_encode(Y_train,cat)
Y_train = np.asarray(Y_train)
X_test_names = test_labels["Id"].values
X_test  = image_set_generator(X_test_names,test_path,'jpg')

#X_train =  preprocessing.scale(X_train)
#X_test = preprocessing.scale(X_test)

In [6]:
#encoder/decoder, quitar comentarios solo en caso de prueba. SPOILER: FUNCIONAN!
#print(Y_train)
#Y_train=cat_encode(Y_train,cat)
#Y_train=cat_decode(Y_train,cat)
#print(Y_train)

# Implementando el modelo

In [7]:
model = Sequential()
model.add(Conv2D(32,(2,2), padding='same', input_shape=(128,128,3)))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(16, (2,2), padding='same'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (2,2), padding='valid'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))


model.add(Conv2D(32, (2,2), padding='valid'))
model.add(BatchNormalization())
model.add(LeakyReLU(alpha=0.1))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())

model.add(Dense(128))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dense(128))
model.add(LeakyReLU(alpha=0.1))
model.add(BatchNormalization())
model.add(Dropout(0.15))
model.add(Dense(10,activation='softmax'))




In [8]:
model.summary()

# Plot the model
plot_model(model, to_file='model.png')

# Display the image
data = plt.imread('model.png')
plt.imshow(data)
plt.show()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 128, 128, 32)      416       
_________________________________________________________________
batch_normalization_1 (Batch (None, 128, 128, 32)      128       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 128, 128, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 16)        2064      
_________________________________________________________________
batch_normalization_2 (Batch (None, 64, 64, 16)        64        
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 64, 64, 16)       

OSError: `pydot` failed to call GraphViz.Please install GraphViz (https://www.graphviz.org/) and ensure that its executables are in the $PATH.

## Callbacks y otras configuraciones para compiler

In [9]:
#########################################           CALLBACKS           #########################################
callback1 = EarlyStopping(monitor='loss', patience=3)
callback2 = ModelCheckpoint(
    filepath="./",
    save_weights_only=False,
    monitor='val_categorical_accuracy',
    mode='max',
    save_best_only=True)
#########################################           OPTIMIZER           #########################################
opt = tf.keras.optimizers.SGD(learning_rate=0.1, momentum=0.015, nesterov=True, name='SGD')
########################################           LOSS FUCTION           #######################################

#########################################           Metrics           #########################################


## Compilado y entrenamiento de nuestro modelo

In [10]:
model.compile(loss="binary_crossentropy",optimizer=opt,metrics=["categorical_accuracy",'accuracy'])
train=model.fit(X_train,Y_train,epochs=30,validation_split=0.3,batch_size=128,shuffle=True,callbacks=[callback1,callback2])

Train on 6448 samples, validate on 2764 samples
Epoch 1/30
3200/6448 [=============>................] - ETA: 1:05 - loss: 0.3163 - categorical_accuracy: 0.2934 - accuracy: 0.8961

KeyboardInterrupt: 

In [ ]:
history=train.history
plt.plot(history['loss'])
# Plot the validation loss
plt.plot(history['val_loss'])

# Show the figure
plt.show()

In [ ]:
auto_predict(X_test,test_labels,cat,True)